In [ ]:
!pip install openpyxl -q

In [ ]:
import pandas as pd
from utils.preprocessing import preprocess_func
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import chain
from nltk.corpus import stopwords

In [ ]:
lemmatize = True
preprocess = preprocess_func(lemmatize=lemmatize)

In [ ]:
df = pd.read_csv('data/processed/row_df_fi.csv', index_col=0)
df.head()

In [ ]:
df['answer'] = df['answer'].apply(preprocess)

In [ ]:
all_words_set = list(set([token for a in df['answer'] for token in a.split()]))

### Rajataan data haluttuun organisaatioon

In [ ]:
org1 = df['organisaatio1'].value_counts().index[1]

is_org1 = df['organisaatio1'] == org1 if org1 else True

# org2 = df.loc[is_org1, 'organisaatio2'].value_counts().index[1]
org2 = None

is_org2 = df['organisaatio2'] == org2 if org2 else True

print(f'organisaatio1: {org1}\norganisaatio2: {org2}')

df_sel = df[is_org1 & is_org2]

print(len(df_sel))

df_sel.head()

### Sanojen frekvenssin analysointi

In [ ]:
_stop = ['esim', 'redacted', 'url', 'mm', 'osata', 'kehittää', 'työ', 'taito', 'kehittyä', 'oppia', 'liittyvä', 'osaaminen', 'käyttö', 'lisätä', 'haluta']

STOP = set(stopwords.words('finnish') 
           + open('data/external/stopwords.txt').read().splitlines()
           + _stop)

In [ ]:
all_sel_words = [a for ans in df_sel['answer'] for a in ans.split()]
all_sel_words_no_stop = [a for a in all_sel_words if a not in STOP]
c = Counter(all_sel_words_no_stop)

In [ ]:
c.most_common(n=20)

### Organisaatioille ominaiset sanat (tf-idf)

In [ ]:
# 1. select the columns we are interested in
# 2. drop nan rows (org not specified)
# 3. group by organization
# 4. concatenate all documents

g = df[['organisaatio1', 'organisaatio2', 'answer']]\
.dropna()\
.groupby(['organisaatio1', 'organisaatio2'])\
.agg(lambda s: ' '.join(chain(*s.str.split())))

tfidf = TfidfVectorizer()
M = tfidf.fit_transform(g['answer'])

# find row index for the selected org1 and org2 pair
def org_eq(idx, orgs):
    pairs = zip(idx, orgs)

    # returns true only if all organizations in `idx` and `orgs` are the same
    # and does not do the comparison if element in `orgs` is not truthy
    return all(idx_org == org if org else True for idx_org, org in pairs)

org_idx = [org_eq(idx, [org1, org2]) for idx in g.index].index(True)


sorted([(word, M[org_idx, word_idx]) for word, word_idx in tfidf.vocabulary_.items() if word not in STOP], 
       key=lambda t: t[1], 
       reverse=True)[:20]

## Word clusters

In [ ]:
cluster_df = pd.read_csv('data/interim/word_clusters_cc.fi.300-lemmatized=True.csv', index_col=0)
# cluster_df.head()

In [ ]:
word2cluster = dict(cluster_df[['word', 'cluster']].itertuples(index=False, name=None))

In [ ]:
cluster_counter = Counter([word2cluster[w] for w in all_sel_words if w in word2cluster])

In [ ]:
top10_clusters = cluster_counter.most_common(n=10)

In [ ]:
# print top-10 words in 10 most common clusters
for i, (cluster, count) in enumerate(top10_clusters):
    _cl_df = cluster_df[cluster_df['cluster'] == cluster]
    print(f'---- [{i}] CLUSTER {cluster} (count {count}) ----')
    print(f"stopword %: {_cl_df['word'].isin(STOP).mean() * 100}")
#     display(_cl_df.sort_values(by='count', ascending=False).set_index('word')[['count']][:10])

## Raportin generointi

In [ ]:
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

### Yleisimmät sanat

In [ ]:
top_n_words = 10

In [ ]:
def top_words_fn(row):
    c = Counter(w for w in row['answer'].split() if w not in STOP)
    
    cols = {'rivien lkm': row['count'], **{f'sana{i}': '' for i in range(1, 1 + top_n_words)}}
    
    for i, (word, count) in enumerate(c.most_common(n=top_n_words), start=1):
        cols[f'sana{i}'] = f'{word} ({count})'
        
    return cols

In [ ]:
sanat1 = df.fillna('.N/A')\
.groupby(['organisaatio1'])\
.agg({'answer': lambda s: ' '.join(chain(*s.str.split())), 'lang': 'count'})\
.rename(columns={'lang' : 'count'})\
.apply(top_words_fn, axis=1, result_type='expand')

sanat1

In [ ]:
# a = df['organisaatio1'] == 'Kaupunginkanslia'
# b = df['organisaatio2'] == 'Kaupunginkanslia'

# for x in df.loc[a & b, 'answer']:
#     print(x)
#     print('-'* 80)

In [ ]:
sanat2 = df.fillna('.N/A')\
.groupby(['organisaatio1', 'organisaatio2'])\
.agg({'answer': lambda s: ' '.join(chain(*s.str.split())), 'lang': 'count'})\
.rename(columns={'lang' : 'count'})\
.apply(top_words_fn, axis=1, result_type='expand')

sanat2

In [ ]:
# sanat3 = df.fillna('.N/A')\
# .groupby(['organisaatio1', 'organisaatio2', 'organisaatio3'])\
# .agg({'answer': lambda s: ' '.join(chain(*s.str.split())), 'lang': 'count'})\
# .rename(columns={'lang' : 'count'})\
# .apply(top_words_fn, axis=1, result_type='expand')

# sanat3

<!-- ### Sanojen määrät tf-idf painoilla

def top_words_tfidf_fn(row):
    idx = row['organisaatio1']

    org_idx = list(g.index == idx).index(True)
    
    words = sorted([(word, M[org_idx, word_idx]) for word, word_idx in tfidf.vocabulary_.items() if word not in STOP], 
           key=lambda t: t[1], 
           reverse=True)[:top_n_words]

    cols = {**{f'sana{i}': w for i, (w, score) in enumerate(words, start=1)}, **row}

    return cols

g = df.fillna('.N/A')\
.groupby(['organisaatio1'])\
.agg({'answer': lambda s: ' '.join(chain(*s.str.split()))})

M = tfidf.fit_transform(g['answer'])


df.fillna('.N/A')\
.groupby(['organisaatio1'])\
.count()\
.drop(columns=[col for col in df.columns if col not in ['organisaatio1', 'lang']])\
.rename(columns={'lang': 'rivien lkm'})\
.reset_index()\
.apply(top_words_tfidf_fn, axis=1, result_type='expand')\
.set_index('organisaatio1') -->

### Sanojen klusterit

In [ ]:
top_n_clusters = 10
top_n_cluster_words = 10

In [ ]:
def word_clusters_fn(row):
    
    cols = {'rivien lkm': row['count'], **{f'klusteri{i}': '' for i in range(1, 1 + top_n_clusters)}}
    
    c = Counter(word2cluster[w] for w in row['answer'].split() if w not in STOP and w in word2cluster)
    
    for i, (cluster, count) in enumerate(c.most_common(n=top_n_clusters), start=1):
        _cl_df = cluster_df[cluster_df['cluster'] == cluster]
        cluster_words = ', '.join(_cl_df.sort_values('count', ascending=False)[:top_n_cluster_words]['word'])
#         cols[f'klusteri{i}'] = f'lkm: {count}, klusteri: {cluster}, sanat: {cluster_words}'
        cols[f'klusteri{i}'] = cluster_words
        
    return cols

In [ ]:
klusterit1 = df.fillna('.N/A')\
.groupby(['organisaatio1'])\
.agg({'answer': lambda s: ' '.join(chain(*s.str.split())), 'lang': 'count'})\
.rename(columns={'lang' : 'count'})\
.apply(word_clusters_fn, axis=1, result_type='expand')

klusterit1

In [ ]:
klusterit2 = df.fillna('.N/A')\
.groupby(['organisaatio1', 'organisaatio2'])\
.agg({'answer': lambda s: ' '.join(chain(*s.str.split())), 'lang': 'count'})\
.rename(columns={'lang' : 'count'})\
.apply(word_clusters_fn, axis=1, result_type='expand')

klusterit2

In [ ]:
# klusterit3 = df.fillna('.N/A')\
# .groupby(['organisaatio1', 'organisaatio2', 'organisaatio3'])\
# .agg({'answer': lambda s: ' '.join(chain(*s.str.split())), 'lang': 'count'})\
# .rename(columns={'lang' : 'count'})\
# .apply(word_clusters_fn, axis=1, result_type='expand')

# klusterit3

In [ ]:
# index = df['organisaatio1'].dropna().unique()

# columns = ['rivien_lkm'] + [sana + str(k) for sana, k in zip(['klusteri'] * 10, range(1, 11))]

# r2 = pd.DataFrame(data=None, index=index, columns=columns)

# for org in index:
#     org_df = df[df['organisaatio1'] == org]
    
#     r2.loc[org, 'rivien_lkm'] = len(org_df)
    
#     org_cluster_counter = Counter([word2cluster[w] for words in org_df['answer'].str.split() for w in words if w in word2cluster])
    
# #     print(org_cluster_counter)

#     for i, (cluster, count) in enumerate(org_cluster_counter.most_common(n=10), start=1):
#         _cl_df = cluster_df[cluster_df['cluster'] == cluster]
#         cluster_words = ', '.join(_cl_df.sort_values('count', ascending=False)[:10]['word'])
# #         display(cluster_words)
# #         r2.loc[org, f'klusteri{i}_lkm'] = count
#         r2.loc[org, f'klusteri{i}'] = f'({count}) {cluster_words}'


# #     break
    
# display(r2)

# r2.to_excel('report/report.xlsx', sheet_name='klusterit')